# Paper

#### Import Functions

In [ ]:
#%matplotlib inline

#import pyvttbl as pt
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.api as sm

import statsmodels.formula.api as smf

import numpy as np
import sklearn
import pandas as pd
#import preprocess2
#import prepare
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from scipy import stats
import warnings
from scipy.stats.mstats import zscore
import matplotlib
#matplotlib.use('TkAgg')  
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as hac
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import cross_val_score
import mne
import imageio
import datetime as dt
import os 


#### Loading the Data

In [ ]:
folder='./mb'
behav_dir='./mb/VIT/Verhaltensdaten'
output_filename_behav='./mb/mb_behav_new.csv'

folder='./mb'
EMG_dir='./mb/VIT/EMG'
output_filename_EMG='./mb/mb_EMG.csv'

video_dir='./mb/VIT/videos'
output_filename_video='./mb/mb_video.csv'

def remove():
    to_delete=[output_filename_behav, output_filename_video, output_filename_EMG]

    for files in to_delete:
        try:
            os.remove(os.path.join(files))
        except OSError:
            pass
        
remove()

In [ ]:
def read_EMG(output_filename_EMG, EMG_dir):
    outputf=open(output_filename_EMG, "a") # create and open the outputfile

    # input folder and list of input files 
    EMG_files = os.listdir(EMG_dir)
        
    #go through all input files
    for filecount, filename in enumerate(EMG_files):
        path=os.path.join(EMG_dir, filename)
        inputf=open(path)
        for c, line in enumerate(inputf): # write all lines of each file into the final one  
            if (filecount==0) & (c==8):
                outputf.write('vpn'   + '\t' + 'frame' + '\t' + 'time' + '\t' + 'delta' + '\t' + line)
            if (c==2):
                time=line
            if (c==4):
                delta=line #print line #4 delts                 
            if(c>8):
                outputf.write(repr(filename) + '\t' + repr(c) + '\t' + repr(time) + '\t' + repr(delta) + '\t'  + line)
        inputf.close() 
    outputf.close()

read_EMG(output_filename_EMG, EMG_dir)

## Preprocessing EMG

In [ ]:
def get_EMG_datafile(filename):
    emg_data=pd.read_csv(filename, sep="\t",  parse_dates=True, na_values=['?'])   

    emg_data.columns = ['vpn', 'frame', 'time', 'delta', 'trigger', 'SCL', 'SCR', 'EMG1',
       'EMG2', 'EMG3', 'Unnamed']
    emg_data['vpn']=emg_data['vpn'].str[4:6] 
    emg_data['timestamp']=pd.to_numeric(emg_data['frame'])-8
    emg_data['rec_date']=pd.to_datetime(emg_data['time'].str[22:42])
    emg_data['rec_delta']=pd.to_numeric(emg_data['delta'].str[22:26])
    emg_data=emg_data.drop(columns=['Unnamed', 'frame', 'time', 'delta'])
    emg_data=emg_data.rename(columns={'EMG1': 'ZYG', 'EMG2': 'COR', 'EMG3' : 'LEV'}) #name muscles
    return emg_data


def get_data_of_vpn(vpn_emg, vpn):
    vpn_emg=vpn_emg[vpn_emg.vpn==vpn].reset_index(drop=True) #get participants data
    start=vpn_emg.rec_date[1] #get recording date
    lines=len(vpn_emg) #get number of records
    vpn_emg.loc[:,'time'] = pd.date_range(start, periods=lines, freq='0.05S') # get sampling indec
    vpn_emg.loc[:,'vpn'] = pd.to_numeric(vpn) # get vpn
    return vpn_emg

def get_conv_parts_and_times(vpn_emg):
    vpn_emg.loc[:, 'conversation_start']=np.max(vpn_emg[vpn_emg.trigger==100]['timestamp'])
    vpn_emg.loc[:,'neutral_speaker_start']=np.max(vpn_emg[vpn_emg.trigger==102]['timestamp'])
    vpn_emg.loc[:,'neutral_proband_start']=np.max(vpn_emg[vpn_emg.trigger==103]['timestamp'])
    vpn_emg.loc[:,'joy_speaker_start']=np.max(vpn_emg[vpn_emg.trigger==104]['timestamp'])
    vpn_emg.loc[:,'joy_proband_start']=np.max(vpn_emg[vpn_emg.trigger==105]['timestamp'])
    vpn_emg.loc[:,'disgust_speaker_start']=np.max(vpn_emg[vpn_emg.trigger==106]['timestamp'])
    vpn_emg.loc[:,'disgust_proband_start']=np.max(vpn_emg[vpn_emg.trigger==107]['timestamp'])
    
    vpn_emg['pretesting']=(vpn_emg.timestamp<vpn_emg.conversation_start)
    vpn_emg['intro']=(vpn_emg.timestamp>=vpn_emg.conversation_start) & (vpn_emg.timestamp<vpn_emg.neutral_speaker_start)
    
    vpn_emg['neutral_speaker']=(vpn_emg.timestamp>=vpn_emg.neutral_speaker_start) & (vpn_emg.timestamp<vpn_emg.neutral_proband_start)
    vpn_emg['neutral_proband']=(vpn_emg.timestamp>=vpn_emg.neutral_proband_start) & (vpn_emg.timestamp<vpn_emg.joy_speaker_start)
    vpn_emg['joy_speaker']=(vpn_emg.timestamp>=vpn_emg.joy_speaker_start) & (vpn_emg.timestamp<vpn_emg.joy_proband_start)
    vpn_emg['joy_proband']=(vpn_emg.timestamp>=vpn_emg.joy_proband_start) & (vpn_emg.timestamp<vpn_emg.disgust_speaker_start)

    vpn_emg['disgust_speaker']=(vpn_emg.timestamp>=vpn_emg.disgust_speaker_start) & (vpn_emg.timestamp<vpn_emg.disgust_proband_start)
    vpn_emg['disgust_proband']=(vpn_emg.timestamp>=vpn_emg.disgust_proband_start)

    return vpn_emg

def convert_conv_times_in_daytime(vpn_emg):
    conversation_start_points=['conversation_start', 'neutral_speaker_start',
                               'neutral_proband_start','joy_speaker_start','joy_proband_start',
                               'disgust_speaker_start', 'disgust_proband_start']
    vpn_emg=vpn_emg.reset_index(drop=True)
    for conv in conversation_start_points:
        timemark_in_s=pd.Timedelta((vpn_emg.iloc[0,:].rec_delta*vpn_emg.iloc[0,:][conv]), unit='s')
        vpn_emg[(conv+'_dt')]=vpn_emg.iloc[0,:].time+timemark_in_s

    return vpn_emg

 # if downsampled=2
def cut_start(df):
    seconds=1.5
    samples_persec=20
    delete=(pd.to_datetime(df.time)<(pd.to_datetime(df.reset_index(drop=True).conversation_start_dt[0])-pd.Timedelta(seconds=seconds)))
    cut_df=df.loc[delete==False].reset_index(drop=True) #loc
    return cut_df

def cut_end(df):
    seconds=42
    delete= pd.to_datetime(df.time)> (pd.to_datetime(df.reset_index(drop=True).disgust_proband_start_dt[0])+pd.Timedelta(seconds=seconds))
    cut_df=df.loc[delete==False].reset_index(drop=True) #loc added
    return cut_df


def outlier_trials(vpn_emg, outl):
    muscles=['ZYG', 'COR', 'LEV']
    for m in muscles:
        
        values=vpn_emg[m]
        vpn_mean=np.nanmean(values, axis=0)
        vpn_std=np.nanstd(values, axis=0)

        if outl=='sd':
            error=np.abs(values-vpn_mean) # es ist unsinn die absoluten Werte der Differenz zu nehmen!!
            out_ind=(np.abs(values-vpn_mean))>(3*vpn_std)

        if outl=='iq':
            quartile_1, quartile_3 = np.percentile(values, [25, 75])
            iqr = quartile_3 - quartile_1
            lower_bound = quartile_1 - (iqr * 3)
            upper_bound = quartile_3 + (iqr * 3)
            out_ind=(values > upper_bound) | (values < lower_bound)

        values[out_ind]=np.nan  
 
        vpn_emg[m]=values.fillna(method='ffill')
        
    return vpn_emg   


def dummycode_conv_parts(df):
    conversation_parts=['pretesting', 'intro', 'neutral_speaker', 'neutral_proband', 'joy_proband',
               'joy_speaker', 'disgust_speaker', 'disgust_proband']

    #round them to avoid half/conversations !
    df[conversation_parts]=np.round(df[conversation_parts])
              
    #categorical encoding
    dummies=df[conversation_parts]
    df["conversation"] = dummies.idxmax(axis=1)
    
    #include a counter of Frames from the Begininng of the Interaction
    df['counter']=df.groupby(['vpn', 'conversation']).cumcount() 
    
    return df


def zscore_muscles(vpn_emg):
    muscles=['ZYG', 'COR', 'LEV']
    for m in muscles:
        vpn_emg[(m+'_z')]=zscore(vpn_emg[m])
        
    return vpn_emg

def baseline_calculation(vpn_emg, basewindow):
    
    vpn_emg=vpn_emg.reset_index()
    vpn_emg['index']=pd.to_numeric(vpn_emg['index'])
    
    muscles=['ZYG', 'COR', 'LEV']
    for con in np.unique(vpn_emg.conversation):
        if (con!='pretesting'): 
            ## achtung das wird für jeden Trigger gemacht und nicht nur einmal !!!
            
            trigger_index=np.int(vpn_emg[(vpn_emg.conversation==con) & (vpn_emg.counter==0)]['index'])
            base_index=np.int(trigger_index-basewindow)


            for m in muscles:
                base_value=np.nanmean(vpn_emg.iloc[(base_index):(trigger_index)][m])
                vpn_emg.loc[(vpn_emg.conversation==con), ('base_' + str(m))]=base_value
                
            #calculate number of broken trials
            vpn_emg['broken']=sum(vpn_emg[muscles].mean(axis=1).isnull())
            vpn_emg.iloc[(base_index):(trigger_index)]['conversation']='base' 
            
    return vpn_emg

def baseline_correction(vpn_emg):
    muscles=['ZYG', 'COR', 'LEV']
    for m in muscles:
        vpn_emg[(m+'_basecorrected')]=vpn_emg[m]-vpn_emg[('base_' + str(m))]
        #alternative: vpn_emg[(m+'_basecorrected')]=vpn_emg[m]/vpn_emg[('base_' + str(m))]
    return vpn_emg

In [ ]:
emg_data=get_EMG_datafile(output_filename_EMG)
#emg_data=final


prepro_emgs = pd.DataFrame()
# accelerati


for vpn in np.unique(emg_data.vpn):
    #get information
    vpn_emg=get_data_of_vpn(emg_data, vpn)

    vpn_emg_with_times=get_conv_parts_and_times(vpn_emg)    
    vpn_emg_with_dt_times=convert_conv_times_in_daytime(vpn_emg_with_times)

    #base sind -100 downsampled
    vpn_emg_with_dt_timest_old=vpn_emg_with_dt_times.copy()
    
    vpn_emg_with_times_firstcut=cut_start(vpn_emg_with_dt_times)
    print vpn_emg_with_dt_timest_old.equals(vpn_emg_with_times_firstcut)
    
    vpn_emg_with_times_firstcut_old=vpn_emg_with_times_firstcut.copy()  
    vpn_emg_with_times_cutted=cut_end(vpn_emg_with_times_firstcut)
    print vpn_emg_with_times_firstcut_old.equals(vpn_emg_with_times_cutted)
    
    vpn_emg_with_times_cutted_old=vpn_emg_with_times_cutted.copy()
    vpn_emg_with_times_without_artefacts=outlier_trials(vpn_emg_with_times_cutted, 'sd')
    print vpn_emg_with_times_without_artefacts.equals(vpn_emg_with_times_cutted_old)
    #outlier-detection
    
    #downsampling of EMG
    sample_rate=500
    downsampled_emg=vpn_emg_with_times_without_artefacts.resample((str(sample_rate)+'ms'), on='time').mean() #downsample
    downsampled_emg=downsampled_emg.reset_index() 
     
    #conversation_time_formating
    downsampled_emg_dt=convert_conv_times_in_daytime(downsampled_emg)
    downsampled_emg_dt_conv=dummycode_conv_parts(downsampled_emg_dt)

    #Baseline-Calculation
    basewindow_in_seconds=2
    basewindow=(1000/sample_rate)*basewindow_in_seconds
    downsampled_emg_dt_conv_base=baseline_calculation(downsampled_emg_dt_conv, basewindow)
    
    #Baseline-Correction
    downsampled_emg_dt_conv_base_old=downsampled_emg_dt_conv_base.copy()
    prepro_emg=baseline_correction(downsampled_emg_dt_conv_base)
    print downsampled_emg_dt_conv_base_old.equals(prepro_emg)
    
    #Zstandardisation within subject for each muscle [does it make sense to z-standardise?]
    #prepro_emg_z=zscore_muscles(prepro_emg) #['seperate variables']
    
    prepro_emg=prepro_emg[prepro_emg.conversation!='pretesting']
    
    #Combine VPN with whole data set
    prepro_emgs=pd.concat([prepro_emgs, prepro_emg], axis=0)
    
    
final_emg=pd.DataFrame(prepro_emgs)#, columns=emg_columns)

In [ ]:
final_emg.to_csv(output_filename_EMG)

In [ ]:
output_filename_EMG

## Read Video-Information and Task-Times and Combine it wih EMG-Data

In [ ]:
def read_behavioral(output_filename_behav, behav_dir):
    outputf=open(output_filename_behav, "a") # create and open the outputfile

    # input folder and list of input files 
    behav_files = os.listdir(behav_dir)

    try:
        os.remove(os.path.join(behav_dir, ".DS_Store"))
    except:
        print("ok")
        
    #go through all input files
    for filecount, filename in enumerate(behav_files):
        path=os.path.join(behav_dir, filename)
        inputf=open(path)
        print path
        if filecount>0:
            inputf.next()
        for c, line in enumerate(inputf): # write all lines of each file into the final one        
            if (filecount==0) & (c==0):
                outputf.write('vpn' + ',' + line)
            else:
                outputf.write(repr(filename) + ',' + line)

        inputf.close() 
    outputf.close()

    
def read_video_info(video_dir, output_filename_video):
    
    outputf=open(output_filename_video, "a") # create and open the outputfile
    video_files = os.listdir(video_dir)

    #go through all input files
    for filecount, filename in enumerate(video_files):
        path=os.path.join(video_dir, filename)

        vid = imageio.get_reader(path,  'ffmpeg')

        num_frames=vid._meta['nframes']     # number of frames in video
        fps = vid._meta['fps']             # framerate of video

        #created = dt.datetime.fromtimestamp(os.stat(path).st_birthtime)
        created = dt.datetime.fromtimestamp(os.stat(path).st_mtime) #notwendig fürProbanden 80-83

        
        duration= num_frames/fps
        start=created - pd.Timedelta(seconds=duration)

        if filecount==0:
            outputf.write('vpn' + ',' + 'start' + ',' + 'end' + ',' + 'num_frames' + ',' + 'fps' + '\n')

        outputf.write(repr(filename) + ',' + str(start) + ',' + str(created) +  ',' + str(num_frames) +  ',' + str(fps) + '\n')

    outputf.close()
    
    
def merge_behav_video(output_filename_behav, output_filename_video):
        
    behav_data=pd.read_csv(output_filename_behav, sep=',', na_values=['?'])   
    
    #load video-information-file
    video_data=pd.read_csv(output_filename_video, sep=',',  parse_dates=True, na_values=['?'])   

    columns=['vpn', 'timer_neutral', 'timer_neutral_end', 'timer_joy',            
                  'timer_joy_end', 'timer_disgust', 'timer_disgust_end']       
    
    behav_data.vpn=behav_data.vpn.str[4:6]
    video_data.vpn=video_data.vpn.str[4:6]

    behav_video_data=pd.merge(behav_data, video_data, on='vpn')
    behav_video_data.rename(columns=lambda x: x.replace(" ", ""), inplace=True) # exclude whitespaces

    return behav_video_data


def merge_all_together(behav_video_data, output_filename_EMG):
    
    #load feature_data
    emg_data=pd.read_csv(output_filename_EMG, sep=",",  parse_dates=True, na_values=['?'])   
    #emg_data['vpn']=emg_data['vpn'].str[4:6] 
    behav_video_data['vpn']=pd.to_numeric(behav_video_data.vpn)
    # Merge together the behavioral/video-data with the video-feature.data
    final=pd.merge(behav_video_data, emg_data, on='vpn')
    final.rename(columns=lambda x: x.replace(" ", ""), inplace=True) # exclude whitespaces
    # drop columns
    
    #CHECKS!!!
    print len(emg_data)
    print len(behav_video_data)
    print len(final)
    
    print len(set(behav_video_data.vpn))
    print len(set(emg_data.vpn))#values
    
    print len(set(behav_video_data.vpn))==len(set(emg_data.vpn))#values
    #ACHTUNG NUMBER 53 war  falsch gelabelt (einmal 33, einmal 11 --> korrekt ist 33) 
    
    return final


In [ ]:
read_behavioral(output_filename_behav, behav_dir)

read_video_info(video_dir, output_filename_video)

behav_video_data=merge_behav_video(output_filename_behav, output_filename_video)


In [ ]:
final=merge_all_together(behav_video_data, output_filename_EMG)

## Combination with Questionnaire Data

In [ ]:
def demographic(df_without_demographics):
    demograph=pd.read_csv('./mb/demographic_data.csv', sep=',', na_values=['?']) 
    
    demograph['vpn']=demograph.VPN.astype(int)
    df_without_demographics['vpn']=df_without_demographics.vpn.astype(int)
      
   
    df_with_demographics=pd.merge(df_without_demographics, demograph, on='vpn')
    return df_with_demographics

def online(df_without_online):
    online=pd.read_csv('./mb/online.csv', sep=';', na_values=['?']) 
    
    online['vpn']=online.VPN.astype(int)
    online['pizza']=online.EV01_01
    online['hering']=online.EV01_02
    
    df_without_online['vpn']=df_without_online.vpn.astype(int)
    
    df_with_online=pd.merge(df_without_online, online[['vpn', 'pizza', 'hering']], on='vpn')
    return df_with_online

In [ ]:
final_plus_demographics=demographic(final)
final_plus_demographics_and_online=online(final_plus_demographics)

In [ ]:
df=final_plus_demographics_and_online

In [ ]:
df.to_csv('final_df_mb.csv')

## Checks

hier stimmt noch etwas nicht: Die Colums existieren nicht mehr. Nochmal überarbeiten!

### Timing

In [ ]:
#final=final[final.vpn!=26]
def checks(final):
    print 'intro'
    print np.min(final['neutral_speaker_start']-final['conversation_start']) #3766.0
    print np.max(final['neutral_speaker_start']-final['conversation_start']) #3807    #10485.0 vpn (26)

    print 'neutral_speaker'
    print np.min(final['neutral_proband_start']-final['neutral_speaker_start']) #554.0
    print np.max(final['neutral_proband_start']-final['neutral_speaker_start']) #557 
    print 'neutral_proband'
    print np.min(final['neutral_proband_start']-final['neutral_speaker_start']) 
    print np.max(final['neutral_proband_start']-final['neutral_speaker_start']) 

    print 'joy_speaker'
    print np.min(final['joy_speaker_start']-final['neutral_proband_start']) 
    print np.max(final['joy_speaker_start']-final['neutral_proband_start'])  
    print 'joy_proband'
    print np.min(final['joy_proband_start']-final['joy_speaker_start']) 
    print np.max(final['joy_proband_start']-final['joy_speaker_start']) 

    print 'disgust_speaker'
    print np.min(final['disgust_speaker_start']-final['joy_proband_start']) 
    print np.max(final['disgust_speaker_start']-final['joy_proband_start']) 

    print 'disgust_proband'
    print np.min(final['disgust_proband_start']-final['disgust_speaker_start']) 
    print np.max(final['disgust_proband_start']-final['disgust_speaker_start']) 

    print 'end'
    print np.max(final['disgust_speaker_start'].reset_index(drop=True)) 
    
    for con in np.unique(df.conversation):
    print con
    for v in np.unique(df.vpn):
        print len(df[df.conversation==con][df.vpn==v])

#multiplicated with 0.05 --> sec.
checks(df)


#### Exclusion based on Checks

In [ ]:
old_final=df.copy()
df=df[df.vpn!=26] #  Probleme mit Lautsprecher/Mikro
df=df[df.vpn!=33] # hat VIT erst nicht gemacht - hier dürfte sonst erst der zweite Durchgang gewertet werden

df=df[df.conversation=='base'].reset_index()

In [ ]:
checks(df)

In [ ]:
def prepare(vpn_emg):
    
    muscles=['ZYG', 'COR', 'LEV']
    for m in muscles:
        vpn_emg[(m+'_mean')]=np.nanmean(vpn_emg[m])    
        vpn_emg[(m+'_max')]=np.max(vpn_emg[m])
        vpn_emg[(m+'_min')]=np.min(vpn_emg[m])  
        vpn_emg[(m+'_var')]=np.var(vpn_emg[m])
        vpn_emg[(m+'_skew')]=np.skew(vpn_emg[m])
        vpn_emg[(m+'_kurtosis')]=np.kurtosis(vpn_emg[m])
     
    return
    #hätte man auch über groupby and apply function machen können
    #könnte man auch nochmal nur nach conversation-parts sich anschauen
    #Mimicry-Mesearues: Difference between ZYG-COR

In [ ]:
#RESTE
final=final[(final.timestamp>final.conversation_start)]  
final.loc[((final.conversation=='ekel_proband')&(final.counter>900)), 'deletion']=1
final=final[final.deletion!=1]